In [1]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import gmean, ttest_ind
from statsmodels.stats.multitest import multipletests
import warnings
warnings.filterwarnings('ignore')

In [6]:
data = pd.read_excel("/Users/mortezaabyadeh/Desktop/DEGs.xlsx")

In [7]:
data.head()

,gene_name,CE_S4,CE_S5,CE_S6,CE_S1,CE_S2,CE_S3,lsm5_Young,N2_Young,pvalue,padj
0,clec-9,2.135698,3.296369,3.501935,3170.295523,2698.154104,2483.626116,2.978001,2784.025248,8.713265e-87,7.141582e-85
1,ilys-3,6.407093,16.481844,5.252903,2767.210522,2676.415834,2120.860653,9.380613,2521.495670,4.560293e-148,1.389963e-145
2,C06E4.6,10.678488,4.395158,6.128387,2697.896964,1744.278805,1558.472854,7.067344,2000.216208,7.596204e-106,9.438162e-104
3,scl-13,2.135698,5.493948,0.875484,397.753189,466.938045,480.309579,2.835043,448.333604,4.138026e-43,9.069715e-42
4,WBGene00008842,3.203546,8.790317,4.377419,657.945623,570.412212,780.249738,5.457094,669.535858,1.967473e-68,9.866193e-67


In [8]:
data.info()
data.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21606 entries, 0 to 21605
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   gene_name   21606 non-null  object 
 1   CE_S4       18943 non-null  float64
 2   CE_S5       18917 non-null  float64
 3   CE_S6       19159 non-null  float64
 4   CE_S1       19349 non-null  float64
 5   CE_S2       19780 non-null  float64
 6   CE_S3       19395 non-null  float64
 7   lsm5_Young  21606 non-null  float64
 8   N2_Young    21606 non-null  float64
 9   pvalue      21602 non-null  float64
 10  padj        21606 non-null  float64
dtypes: float64(10), object(1)
memory usage: 1.8+ MB


gene_name        0
CE_S4         2663
CE_S5         2689
CE_S6         2447
CE_S1         2257
CE_S2         1826
CE_S3         2211
lsm5_Young       0
N2_Young         0
pvalue           4
padj             0
dtype: int64

In [5]:
data.shape

(21606, 15)

In [9]:
control_cols = ["CE_S1", "CE_S2", "CE_S3"]
treatment_cols = ["CE_S4", "CE_S5", "CE_S6"]

In [10]:
def filter_rows(row):
    control_nan_count = row[control_cols].isna().sum()
    treatment_nan_count = row[treatment_cols].isna().sum()
    return not ((control_nan_count > 2) or (treatment_nan_count > 2))

filtered_data = data[data.apply(filter_rows, axis=1)]

In [13]:
filtered_data.isna().sum()

(19890, 11)


In [15]:
filtered_data.shape

(19890, 11)

In [16]:
def impute_group(group):
    imputer = IterativeImputer(estimator=RandomForestRegressor(), max_iter=10, random_state=0)
    imputed_group = imputer.fit_transform(group)
    return pd.DataFrame(imputed_group, columns=group.columns, index=group.index)

control_imputed = impute_group(filtered_data[control_cols])
filtered_data[control_cols] = control_imputed

treatment_imputed = impute_group(filtered_data[treatment_cols])
filtered_data[treatment_cols] = treatment_imputed

In [17]:
filtered_data.isna().sum()

gene_name     0
CE_S4         0
CE_S5         0
CE_S6         0
CE_S1         0
CE_S2         0
CE_S3         0
lsm5_Young    0
N2_Young      0
pvalue        4
padj          0
dtype: int64

In [19]:
print(filtered_data.head(10))
print(filtered_data.tail(10))

        gene_name      CE_S4      CE_S5     CE_S6        CE_S1        CE_S2  \
0          clec-9   2.135698   3.296369  3.501935  3170.295523  2698.154104   
1          ilys-3   6.407093  16.481844  5.252903  2767.210522  2676.415834   
2         C06E4.6  10.678488   4.395158  6.128387  2697.896964  1744.278805   
3          scl-13   2.135698   5.493948  0.875484   397.753189   466.938045   
4  WBGene00008842   3.203546   8.790317  4.377419   657.945623   570.412212   
5         ZK355.3   2.135698   1.098790  1.750968   154.622553   150.428830   
6          rgba-1   1.067849   2.197579  1.750968   155.688916   135.646806   
7         col-108   4.271395   4.395158  2.626452   296.448757   399.114642   
8          gst-34   5.339244   6.592738  9.630322   874.417198   540.848164   
9         C13A2.4   5.339244   2.197579  7.879355   343.368705   273.032674   

         CE_S3  lsm5_Young     N2_Young         pvalue           padj  
0  2483.626116    2.978001  2784.025248   8.713265e-87   7

In [20]:
filtered_data.shape

(19890, 11)

In [24]:
dat = filtered_data.dropna(subset=["gene_name"])

In [25]:
dat.shape

(19890, 11)

In [26]:
dat['Mean_lsm5'] = gmean(dat[treatment_cols], axis=1)
dat['Mean_N2'] = gmean(dat[control_cols], axis=1)

In [27]:
dat["Fold Change"] = dat['Mean_lsm5']/dat['Mean_N2']
dat['log2_Fold Change'] = np.log2(dat['Fold Change'])

In [28]:
dat.head()

,gene_name,CE_S4,CE_S5,CE_S6,CE_S1,CE_S2,CE_S3,lsm5_Young,N2_Young,pvalue,padj,Mean_lsm5,Mean_N2,Fold Change,log2_Fold Change
0,clec-9,2.135698,3.296369,3.501935,3170.295523,2698.154104,2483.626116,2.978001,2784.025248,8.713265e-87,7.141582e-85,2.910457,2769.603325,0.001051,-9.894218
1,ilys-3,6.407093,16.481844,5.252903,2767.210522,2676.415834,2120.860653,9.380613,2521.495670,4.560293e-148,1.389963e-145,8.216535,2504.393927,0.003281,-8.251716
2,C06E4.6,10.678488,4.395158,6.128387,2697.896964,1744.278805,1558.472854,7.067344,2000.216208,7.596204e-106,9.438162e-104,6.601006,1942.883836,0.003398,-8.201298
3,scl-13,2.135698,5.493948,0.875484,397.753189,466.938045,480.309579,2.835043,448.333604,4.138026e-43,9.069715e-42,2.173823,446.818754,0.004865,-7.683312
4,WBGene00008842,3.203546,8.790317,4.377419,657.945623,570.412212,780.249738,5.457094,669.535858,1.967473e-68,9.866193e-67,4.976812,664.055151,0.007495,-7.059937


In [39]:
dat[treatment_cols] = dat[treatment_cols].apply(pd.to_numeric, errors='coerce')
dat[control_cols] = dat[control_cols].apply(pd.to_numeric, errors='coerce')
dat1 = dat.copy()
dat1.head()
control_values = pd.to_numeric(row[control_cols], errors='coerce').dropna()
treatment_values = pd.to_numeric(row[treatment_cols], errors='coerce').dropna()

In [42]:
t_stats = []
p_values = []

# Compute t-test statistics and p-values for each row
for index, row in dat1.iterrows():
    control_values = pd.to_numeric(row[control_cols], errors='coerce').dropna()
    treatment_values = pd.to_numeric(row[treatment_cols], errors='coerce').dropna()
    
    # Ensure there are enough values to perform the t-test
    if len(control_values) > 1 and len(treatment_values) > 1:
        t_stat, p_value = ttest_ind(control_values, treatment_values, equal_var=False)
    else:
        t_stat, p_value = np.nan, np.nan  # Not enough data to perform t-test
    
    t_stats.append(t_stat)
    p_values.append(p_value)

# Assign the results to new columns in dat1
dat1["T-Stat"] = t_stats
dat1["P-Value"] = p_values

In [43]:
dat1.head(10)

,gene_name,CE_S4,CE_S5,CE_S6,CE_S1,CE_S2,CE_S3,lsm5_Young,N2_Young,pvalue,padj,Mean_lsm5,Mean_N2,Fold Change,log2_Fold Change,T-Stat,P-Value
0,clec-9,2.135698,3.296369,3.501935,3170.295523,2698.154104,2483.626116,2.978001,2784.025248,8.713265e-87,7.141582e-85,2.910457,2769.603325,0.001051,-9.894218,13.711799,0.005277
1,ilys-3,6.407093,16.481844,5.252903,2767.210522,2676.415834,2120.860653,9.380613,2521.495670,4.560293e-148,1.389963e-145,8.216535,2504.393927,0.003281,-8.251716,12.432741,0.006392
2,C06E4.6,10.678488,4.395158,6.128387,2697.896964,1744.278805,1558.472854,7.067344,2000.216208,7.596204e-106,9.438162e-104,6.601006,1942.883836,0.003398,-8.201298,5.647195,0.029951
3,scl-13,2.135698,5.493948,0.875484,397.753189,466.938045,480.309579,2.835043,448.333604,4.138026e-43,9.069715e-42,2.173823,446.818754,0.004865,-7.683312,17.388572,0.003208
4,WBGene00008842,3.203546,8.790317,4.377419,657.945623,570.412212,780.249738,5.457094,669.535858,1.967473e-68,9.866193e-67,4.976812,664.055151,0.007495,-7.059937,10.908853,0.008253
5,ZK355.3,2.135698,1.098790,1.750968,154.622553,150.428830,262.447639,1.661818,189.166341,3.270644e-22,3.048277e-21,1.601686,182.760441,0.008764,-6.834218,5.114426,0.036159
6,rgba-1,1.067849,2.197579,1.750968,155.688916,135.646806,207.728826,1.672132,166.354850,1.257597e-21,1.135738e-20,1.601686,163.702261,0.009784,-6.675339,7.665607,0.016574
7,col-108,4.271395,4.395158,2.626452,296.448757,399.114642,393.164803,3.764335,362.909401,3.801723e-44,8.613264e-43,3.666946,359.644958,0.010196,-6.615850,10.791760,0.008460
8,gst-34,5.339244,6.592738,9.630322,874.417198,540.848164,624.199791,7.187435,679.821718,8.715859e-71,4.632705e-69,6.972611,665.844101,0.010472,-6.577342,6.710448,0.021476
9,C13A2.4,5.339244,2.197579,7.879355,343.368705,273.032674,743.770530,5.138726,453.390636,5.625362e-21,4.929647e-20,4.521734,411.596273,0.010986,-6.508210,3.057402,0.092363


In [44]:
p_values = dat1["P-Value"]

adjusted_p_values = multipletests(p_values, method='bonferroni')[1]

dat1["Adjusted_P_Value_Bonferroni"] = adjusted_p_values

In [45]:
p_values = dat1["P-Value"]

adjusted_p_values_bh = multipletests(p_values, method='fdr_bh')[1]

dat1["Adjusted_P_Value_BH"] = adjusted_p_values_bh

In [46]:
sorted_dat1 = dat1.sort_values(by="P-Value")
sorted_dat1.head()

,gene_name,CE_S4,CE_S5,CE_S6,CE_S1,CE_S2,CE_S3,lsm5_Young,N2_Young,pvalue,padj,Mean_lsm5,Mean_N2,Fold Change,log2_Fold Change,T-Stat,P-Value,Adjusted_P_Value_Bonferroni,Adjusted_P_Value_BH
15139,F38B2.6,461.310680,465.886799,457.878051,82.109908,87.822612,79.038285,461.691843,82.990268,3.148922e-52,9.421170e-51,461.680197,82.911188,5.568370,2.477255,-109.296908,4.898312e-08,0.000974,0.000547
15701,C34G6.3,199.687725,204.374869,201.361284,18.128161,21.738270,17.226293,201.807960,19.030908,9.003013e-37,1.589337e-35,201.798660,18.934631,10.657649,3.413817,-93.999311,7.682209e-08,0.001528,0.000547
471,nnt-1,749.629855,724.102360,651.359980,4758.109191,4782.419457,4678.458497,708.364065,4739.662382,1.368983e-166,6.210407e-164,707.113784,4739.453682,0.149197,-2.744706,93.665076,8.245622e-08,0.001640,0.000547
354,fmo-1,1455.477909,1282.287487,1510.209632,12119.209089,12046.479846,11836.489864,1415.991676,12000.726266,1.120672e-216,9.108728e-214,1412.568182,12000.125529,0.117713,-3.086657,97.017765,1.204626e-07,0.002396,0.000599
13659,snr-1,2688.843269,2679.947885,2654.467016,1330.820321,1368.641495,1327.437867,2674.419390,1342.299894,1.083596e-28,1.364603e-27,2674.379636,1342.170770,1.992578,0.994636,-79.532471,3.143000e-07,0.006251,0.000940


In [47]:
sorted_dat1.to_excel("/Users/mortezaabyadeh/Desktop/Modified_DEGs.xlsx", index=False)

In [48]:
sorted_dat1.shape

(19890, 19)

In [49]:
dat2 = sorted_dat1[sorted_dat1["P-Value"] < 0.05]
dat2.head()

,gene_name,CE_S4,CE_S5,CE_S6,CE_S1,CE_S2,CE_S3,lsm5_Young,N2_Young,pvalue,padj,Mean_lsm5,Mean_N2,Fold Change,log2_Fold Change,T-Stat,P-Value,Adjusted_P_Value_Bonferroni,Adjusted_P_Value_BH
15139,F38B2.6,461.310680,465.886799,457.878051,82.109908,87.822612,79.038285,461.691843,82.990268,3.148922e-52,9.421170e-51,461.680197,82.911188,5.568370,2.477255,-109.296908,4.898312e-08,0.000974,0.000547
15701,C34G6.3,199.687725,204.374869,201.361284,18.128161,21.738270,17.226293,201.807960,19.030908,9.003013e-37,1.589337e-35,201.798660,18.934631,10.657649,3.413817,-93.999311,7.682209e-08,0.001528,0.000547
471,nnt-1,749.629855,724.102360,651.359980,4758.109191,4782.419457,4678.458497,708.364065,4739.662382,1.368983e-166,6.210407e-164,707.113784,4739.453682,0.149197,-2.744706,93.665076,8.245622e-08,0.001640,0.000547
354,fmo-1,1455.477909,1282.287487,1510.209632,12119.209089,12046.479846,11836.489864,1415.991676,12000.726266,1.120672e-216,9.108728e-214,1412.568182,12000.125529,0.117713,-3.086657,97.017765,1.204626e-07,0.002396,0.000599
13659,snr-1,2688.843269,2679.947885,2654.467016,1330.820321,1368.641495,1327.437867,2674.419390,1342.299894,1.083596e-28,1.364603e-27,2674.379636,1342.170770,1.992578,0.994636,-79.532471,3.143000e-07,0.006251,0.000940


In [50]:
dat2.shape

(10707, 19)

In [51]:
dat2_sorted = dat2.sort_values(by="Fold Change")
dat2_sorted.head()

,gene_name,CE_S4,CE_S5,CE_S6,CE_S1,CE_S2,CE_S3,lsm5_Young,N2_Young,pvalue,padj,Mean_lsm5,Mean_N2,Fold Change,log2_Fold Change,T-Stat,P-Value,Adjusted_P_Value_Bonferroni,Adjusted_P_Value_BH
0,clec-9,2.135698,3.296369,3.501935,3170.295523,2698.154104,2483.626116,2.978001,2784.025248,8.713265e-87,7.141582e-85,2.910457,2769.603325,0.001051,-9.894218,13.711799,0.005277,1.0,0.024917
1,ilys-3,6.407093,16.481844,5.252903,2767.210522,2676.415834,2120.860653,9.380613,2521.495670,4.560293e-148,1.389963e-145,8.216535,2504.393927,0.003281,-8.251716,12.432741,0.006392,1.0,0.027208
2,C06E4.6,10.678488,4.395158,6.128387,2697.896964,1744.278805,1558.472854,7.067344,2000.216208,7.596204e-106,9.438162e-104,6.601006,1942.883836,0.003398,-8.201298,5.647195,0.029951,1.0,0.065150
16740,Y82E9BL.3,2.152881,1.098790,0.875484,347.634154,226.947541,291.833668,0.658091,288.805121,5.152375e-17,3.673516e-16,1.274660,284.486135,0.004481,-7.802102,8.241859,0.014393,1.0,0.041484
3,scl-13,2.135698,5.493948,0.875484,397.753189,466.938045,480.309579,2.835043,448.333604,4.138026e-43,9.069715e-42,2.173823,446.818754,0.004865,-7.683312,17.388572,0.003208,1.0,0.019581


In [52]:
with pd.ExcelWriter('/Users/mortezaabyadeh/Desktop/Cleaned_DEGs.xlsx', engine='xlsxwriter') as writer:
    dat2_sorted.to_excel(writer, sheet_name='All Data', index=False)

    low_abundance = dat2_sorted[dat2_sorted["log2_Fold Change"] < 0]
    high_abundance = dat2_sorted[dat2_sorted["log2_Fold Change"] > 0]

    low_abundance.to_excel(writer, sheet_name='Low Abundance', index=False)
    high_abundance.to_excel(writer, sheet_name='High Abundance', index=False)

    writer.save()